# `pyLDAvis.sklearn`

Modified from the pyLDAvis scikit-learn example notebook.

In [1]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import re, io, os
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict
import urllib.request

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## Load Voynich Data

Load the Pickle files produced by vms_vectorize.py

In [3]:
def tokenize(data):
    index = defaultdict(str)
    
    with urllib.request.urlopen(data) as file:
        for line in file.read().decode('latin-1').splitlines():
            # pull out takahashi lines
            m = re.match(r'^<(f.*?)\..*;H> +(\S.*)$', line)
            if not m:
                continue

            transcription = m.group(2)
            pg = str(m.group(1))

            # ignore entire line if it has a {&NNN} or {&.} code
            if re.search(r'\{&(\d|\.)+\}', transcription):
                continue

            # remove extraneous chracters ! and %
            s = transcription.replace("!", "").replace("%", "")
            
            # delete all end of line {comments} (between one and three observed)
            # ...with optional line terminator
            # allow 0 occurences to remove end-of-line markers (- or =)
            s = re.sub(r'([-=]?\{[^\{\}]+?\}){0,3}[-=]?\s*$', "", s)

            # delete start of line {comments} (single or double)
            s = re.sub(r'^(\{[^\{\}]+?\}){1,2}', "", s)

            # simplification: tags preceeded by -= are word breaks
            s = re.sub(r'[-=]\{[^\{\}]+?\}', '.', s)

            # these tags are nulls
            # plant is a null in one case where it is just {plant}
            # otherwise (above) it is a word break
            # s = re.sub(r'\{(fold|crease|blot|&\w.?|plant)\}', "", s)
            # simplification: remaining tags in curly brackets
            s = re.sub(r'\{[^\{\}]+?\}', '', s)

            # special case .{\} is still a word break
            s = re.sub(r'\.\{\\\}', ".", s)

            # split on word boundaries
            # exclude null words ('')
            words = [str(w) for w in s.split(".") if w]
            paragraph = ' '.join(words).lstrip()
            
            index[pg] += (paragraph)

    return index


In [4]:
models_path = "./models"

# load in the pickle files of stored models
with open("{}/tfidf_vectorizer.pk".format(models_path), "rb") as f:
    tfidf_vectorizer = pickle.load(f)
with open("{}/vms_tf.pk".format(models_path), "rb") as f:
    vms_tf = pickle.load(f)
with open("{}/vms_mapping.pk".format(models_path), "rb") as f:
    vms_mapping = pickle.load(f)
with open("{}/tf_vectorizer.pk".format(models_path), "rb") as f:
    tf_vectorizer = pickle.load(f)
with open("{}/vms_tfidf.pk".format(models_path), "rb") as f:
    vms_tfidf = pickle.load(f)

num_topics = 6

index = tokenize("https://raw.githubusercontent.com/rachelbari/voynich-topic-modeling/master/data/text16e6.evt")
documents = [index[key] for key in index.keys()]

#newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
#docs_raw = newsgroups.data
#print(len(docs_raw))

/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
# make a dataframe of mapping material
map_df = pd.DataFrame(vms_mapping, columns=['folio'])
print(map_df)
print(vms_mapping)


     folio
0      f1r
1      f1v
2      f2r
3      f2v
4      f3r
..     ...
220  f114r
221  f114v
222  f115r
223  f115v
224  f116r

[225 rows x 1 columns]
['f1r', 'f1v', 'f2r', 'f2v', 'f3r', 'f3v', 'f4r', 'f4v', 'f5r', 'f5v', 'f6r', 'f6v', 'f7r', 'f7v', 'f8r', 'f8v', 'f9r', 'f9v', 'f10r', 'f10v', 'f11r', 'f11v', 'f13r', 'f13v', 'f14r', 'f14v', 'f15r', 'f15v', 'f16r', 'f16v', 'f17r', 'f17v', 'f18r', 'f18v', 'f19r', 'f19v', 'f20r', 'f20v', 'f21r', 'f21v', 'f22r', 'f22v', 'f23r', 'f23v', 'f24r', 'f24v', 'f25r', 'f25v', 'f26r', 'f26v', 'f27r', 'f27v', 'f28r', 'f28v', 'f29r', 'f29v', 'f30r', 'f30v', 'f31r', 'f31v', 'f32r', 'f32v', 'f33r', 'f33v', 'f34r', 'f34v', 'f35r', 'f35v', 'f36r', 'f36v', 'f37r', 'f37v', 'f38r', 'f38v', 'f39r', 'f39v', 'f40r', 'f40v', 'f41r', 'f41v', 'f42r', 'f42v', 'f43r', 'f43v', 'f44r', 'f44v', 'f45r', 'f45v', 'f46r', 'f46v', 'f47r', 'f47v', 'f48r', 'f48v', 'f49r', 'f49v', 'f50r', 'f50v', 'f51r', 'f51v', 'f52r', 'f52v', 'f53r', 'f53v', 'f54r', 'f54v', 'f55r', 'f55v

## Fit Latent Dirichlet Allocation models

Finally, the LDA models are fitted.

In [6]:
# for TF
lda_tf = LatentDirichletAllocation(n_components=num_topics, random_state=0, max_iter=8)#, learning_method="online", learning_offset=20.)
lda_tf.fit(vms_tf)
# for TFIDF
#lda_tfidf = LatentDirichletAllocation(n_components=num_topics, random_state=0, max_iter=8)#, learning_method="online", learning_offset=20.)
#lda_tfidf.fit(vms_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=8,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## Visualizing the models with pyLDAvis

In [7]:
pyLDAvis.sklearn.prepare(lda_tf, vms_tf, tf_vectorizer)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.156912 -0.115019       1        1  32.181026
2     -0.036320  0.038596       2        1  25.063990
0      0.032549 -0.033707       3        1  21.569351
5     -0.247885 -0.083032       4        1  16.520796
3      0.128746  0.040138       5        1   3.134436
1     -0.034002  0.153024       6        1   1.530401, topic_info=     Category        Freq     Term       Total  loglift  logprob
1390  Default  249.000000  qokeedy  249.000000  30.0000  30.0000
1625  Default  381.000000    shedy  381.000000  29.0000  29.0000
1401  Default  243.000000   qokeey  243.000000  28.0000  28.0000
465   Default  545.000000    daiin  545.000000  27.0000  27.0000
318   Default  343.000000     chol  343.000000  26.0000  26.0000
...       ...         ...      ...         ...      ...      ...
1625   Topic6    1.461964    shedy  381.981172  -1.3859  -5.5461
1401   Topic6    1.408222   qokeey  243.674118  -0.9739  -5.5835
133    Topic6    1.355773     chdy  122.988838  -0.3281  -5.6215
255    Topic6    1.403864     chey  301.887894  -1.1912  -5.5866
512    Topic6    1.365693      dar  225.560183  -0.9273  -5.6142

[451 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1         1  0.209894  aiiin
1         2  0.209894  aiiin
1         3  0.577209  aiiin
2         1  0.122306   aiin
2         2  0.349795   aiin
...     ...       ...    ...
1920      1  0.085801  yteey
1920      2  0.214503  yteey
1920      3  0.643509  yteey
1920      6  0.042901  yteey
1929      4  0.880457   ytol

[888 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 1, 6, 4, 2])

In [8]:
#pyLDAvis.sklearn.prepare(lda_tfidf, vms_tfidf, tfidf_vectorizer)

### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [9]:
pyLDAvis.sklearn.prepare(lda_tf, vms_tf, tf_vectorizer, mds='mmds')

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.170144  0.117047       1        1  32.181026
2     -0.096434 -0.070842       2        1  25.063990
0      0.006068  0.067431       3        1  21.569351
5     -0.243216  0.132550       4        1  16.520796
3      0.172301 -0.067990       5        1   3.134436
1     -0.008863 -0.178196       6        1   1.530401, topic_info=     Category        Freq     Term       Total  loglift  logprob
1390  Default  249.000000  qokeedy  249.000000  30.0000  30.0000
1625  Default  381.000000    shedy  381.000000  29.0000  29.0000
1401  Default  243.000000   qokeey  243.000000  28.0000  28.0000
465   Default  545.000000    daiin  545.000000  27.0000  27.0000
318   Default  343.000000     chol  343.000000  26.0000  26.0000
...       ...         ...      ...         ...      ...      ...
1625   Topic6    1.461964    shedy  381.981172  -1.3859  -5.5461
1401   Topic6    1.408222   qokeey  243.674118  -0.9739  -5.5835
133    Topic6    1.355773     chdy  122.988838  -0.3281  -5.6215
255    Topic6    1.403864     chey  301.887894  -1.1912  -5.5866
512    Topic6    1.365693      dar  225.560183  -0.9273  -5.6142

[451 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1         1  0.209894  aiiin
1         2  0.209894  aiiin
1         3  0.577209  aiiin
2         1  0.122306   aiin
2         2  0.349795   aiin
...     ...       ...    ...
1920      1  0.085801  yteey
1920      2  0.214503  yteey
1920      3  0.643509  yteey
1920      6  0.042901  yteey
1929      4  0.880457   ytol

[888 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 1, 6, 4, 2])

In [10]:
pyLDAvis.sklearn.prepare(lda_tf, vms_tf, tf_vectorizer, mds='tsne')

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4      75.338402   0.537533       1        1  32.181026
2      19.047890 -56.690876       2        1  25.063990
0      30.950418  22.691652       3        1  21.569351
5      67.984123 -48.527706       4        1  16.520796
3      -3.837605 -12.674557       5        1   3.134436
1      37.898197 -18.934160       6        1   1.530401, topic_info=     Category        Freq     Term       Total  loglift  logprob
1390  Default  249.000000  qokeedy  249.000000  30.0000  30.0000
1625  Default  381.000000    shedy  381.000000  29.0000  29.0000
1401  Default  243.000000   qokeey  243.000000  28.0000  28.0000
465   Default  545.000000    daiin  545.000000  27.0000  27.0000
318   Default  343.000000     chol  343.000000  26.0000  26.0000
...       ...         ...      ...         ...      ...      ...
1625   Topic6    1.461964    shedy  381.981172  -1.3859  -5.5461
1401   Topic6    1.408222   qokeey  243.674118  -0.9739  -5.5835
133    Topic6    1.355773     chdy  122.988838  -0.3281  -5.6215
255    Topic6    1.403864     chey  301.887894  -1.1912  -5.5866
512    Topic6    1.365693      dar  225.560183  -0.9273  -5.6142

[451 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1         1  0.209894  aiiin
1         2  0.209894  aiiin
1         3  0.577209  aiiin
2         1  0.122306   aiin
2         2  0.349795   aiin
...     ...       ...    ...
1920      1  0.085801  yteey
1920      2  0.214503  yteey
1920      3  0.643509  yteey
1920      6  0.042901  yteey
1929      4  0.880457   ytol

[888 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 1, 6, 4, 2])